# Renaming Files with the same pattern

In [ ]:
import os
import pandas as pd
from openpyxl import load_workbook

In [ ]:

def project_excel_files(directory):
    files_format_region_end = []
    files_format_dataset = []
    
    # Run through all files in folder
    for filename in os.listdir(directory):
        if "REACH_SYR_HSOS_Dataset" in filename:
            files_format_dataset.append(filename)
        else:
            files_format_region_end.append(filename)
            
    return files_format_region_end, files_format_dataset

# Directory path
directory = "C:/Users/caspa/OneDrive/00 DataScientist/Projekt/HSOS/Uni-1"
format_region_end, format_region_first = project_excel_files(directory)

# Date Mapper
month_map = {
    "January": "01", "February": "02", "March": "03",
    "April": "04", "May": "05", "June": "06",
    "July": "07", "August": "08", "September": "09",
    "October": "10", "November": "11", "December": "12"
}

new_filenames_region_end = []
new_filenames_region_first = []

# Format for file with region end
for filename in format_region_end:
    filename = filename.replace(".xlsx", "")
    parts = filename.split("_")
    date = parts[-1]
    region = parts[-3]
    
    # Get month & year
    month = ''.join(filter(str.isalpha, date))
    year = ''.join(filter(str.isdigit, date))
    format_date = f"{month_map[month]}_{year}"
    
    new_filename = f"HSOS_{region}_Dataset_{format_date}.xlsx"
    new_filenames_region_end.append(new_filename)

# Format for file with region first
for filename in format_region_first:
    parts = filename.split("_")
    date = parts[-2]
    region = parts[-1].replace(".xlsx", "")
    
    # Get month & year
    month = ''.join(filter(str.isalpha, date))
    year = ''.join(filter(str.isdigit, date))
    format_date = f"{month_map[month]}_{year}"
    
    new_filename = f"HSOS_{region}_Dataset_{format_date}.xlsx"
    new_filenames_region_first.append(new_filename)

# Rename files in 'format_region_end' group
for old_filename, new_filename in zip(format_region_end, new_filenames_region_end):
    old_filepath = os.path.join(directory, old_filename)
    new_filepath = os.path.join(directory, new_filename)
    os.rename(old_filepath, new_filepath)
    print(f"Renamed: {old_filename} --> {new_filename}")

# Rename files in 'format_region_first' group
for old_filename, new_filename in zip(format_region_first, new_filenames_region_first):
    old_filepath = os.path.join(directory, old_filename)
    new_filepath = os.path.join(directory, new_filename)
    os.rename(old_filepath, new_filepath)
    print(f"Renamed: {old_filename} --> {new_filename}")

# Read all sheets & Create df from all the sheets

In [ ]:
def get_sheet_names(file_path):
    try:
        # Load Excel File
        workbook = load_workbook(file_path, read_only=True)
        # Read Sheetnames
        return workbook.sheetnames
    except Exception as e:
        print(f"Fehler beim Verarbeiten der Datei {file_path}: {e}")
        return []

def create_sheet_names_dataframe(directory):
    data = []
    files = [f for f in os.listdir(directory) if f.endswith(".xlsx")]

    for file in files:
        file_path = os.path.join(directory, file)
        sheet_names = get_sheet_names(file_path)
        if sheet_names:
            data.append([file] + sheet_names)

    # Max Sheet names
    max_sheets = max(len(row) for row in data)

    # Dynamic columnnames
    column_names = ["Dateiname"] + [f"Blattname {i}" for i in range(1, max_sheets)]

    # Create DataFrame
    df = pd.DataFrame(data, columns=column_names)

    return df


# Extracting all columns

In [ ]:
directory = "C:/Users/caspa/OneDrive/00 DataScientist/Projekt/HSOS/Uni-1/"
e_sheet_name = "dataset1"

# Extract columns names
def column_ex(filepath):
    workbook = pd.read_excel(filepath, sheet_name=e_sheet_name)
    return list(workbook.columns)

column_names = []

# Maximum columns number
max_columns = 0

# Iterate through all files in folder
for file in os.listdir(directory):
    file_path = os.path.join(directory, file)  
    try:
        columns = column_ex(file_path)
        # Aktualisieren der maximalen Spaltenanzahl
        max_columns = max(max_columns, len(columns))
        column_names.append({"File": file, **{f"Column_{i+1}": col for i, col in enumerate(columns)}})
        print(f"Datei {file} erfolgreich")
    except Exception as e:
        print(f"Fehler bei Datei {file}: {e}")  

# all files have the max col number
for entry in column_names:
    # Max col number
    file_columns = [entry.get(f"Column_{i+1}") for i in range(max_columns)]
    # Nan if cols dont exist
    file_columns += [None] * (max_columns - len(file_columns))
    entry.update({f"Column_{i+1}": col for i, col in enumerate(file_columns)})

# df
df_col_names = pd.DataFrame(column_names)

# creating comparison file

In [ ]:
df = pd.read_csv("C:/Users/caspa/OneDrive/00 DataScientist/Projekt/HSOS/Columnname_Overview.csv")

data_values = df.drop(columns=["File"])

# Dictionary
value_to_files = {}

# Iterate df
for col in data_values.columns:  # per column
    for index, value in data_values[col].items():  # per row
        if pd.notna(value):  # look at non nan values
            if value not in value_to_files:
                value_to_files[value] = []  
            value_to_files[value].append(df.loc[index, "File"])

# List of all files
all_files = set(df["File"])

# Create df
exploded_data = pd.DataFrame(
    {file: [1 if file in files else 0 for files in value_to_files.values()] for file in all_files},
    index=value_to_files.keys()
)

# Index and Col naming
exploded_data.index.name = "Value"
exploded_data.columns.name = "File"

# Export
exploded_data.to_excel("C:/Users/caspa/OneDrive/00 DataScientist/Projekt/HSOS/Common_col.xlsx")
exploded_data.to_csv("C:/Users/caspa/OneDrive/00 DataScientist/Projekt/HSOS/Common_col.csv")


# working dataframe

In [ ]:
# Load the reference file
reference = pd.read_csv(
    "C:/Users/caspa/OneDrive/00 DataScientist/Projekt/HSOS/columns_from_mapping.csv", 
    sep=";")

# Transpose reference if necessary
ref_columns = reference.columns  # Assuming column names are needed for comparison

directory = "C:/Users/caspa/OneDrive/00 DataScientist/Projekt/HSOS/HSOS DATA/"

# Define the function to get data
def get_data(filepath):
    try:
        # Extract the last 11 characters of the file name (excluding extension)
        file_name = os.path.basename(filepath)
        name_column = file_name[:-5][-11:]  # Remove .xlsx and get last 11 characters
        
        # Read data from the Excel file
        data = pd.read_excel(filepath, sheet_name="dataset1")
        
        # Filter data based on reference columns
        filtered_data = data[[col for col in data.columns if col in ref_columns]].copy()
        
        # Add a new column with the extracted name
        filtered_data.loc[:, "Name"] = name_column
        
        return filtered_data
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return None

# Running through files and getting data
final = []

for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    print(file_path)
    if file_path.endswith(".xlsx"):  # Ensure only Excel files are processed
        data = get_data(file_path)
        if data is not None:
            final.append(data)

# Combine all the collected data
if final:
    df = pd.concat(final, ignore_index=True)
    print("Final DataFrame created successfully.")
    print(df.head())  # Display a preview of the final DataFrame
else:
    print("No valid data was processed.")

df["Year"] = df["Name"].str.split("_").str[0].astype(int)
df["Month"] = df["Name"].str.split("_").str[1].astype(int)
#df["Region"] = df["Name"].str.split("_").str[2]

move_col = df.pop("Year")
df.insert(0, "Year", move_col)

move_col = df.pop("Month")
df.insert(1, "Month", move_col)


df.to_csv("C:/Users/caspa/OneDrive/00 DataScientist/Projekt/HSOS/Final_DF_mapped.csv", index=False, sep=";")

#excel export doesnt work
#df.to_excel("C:/Users/caspa/OneDrive/00 DataScientist/Projekt/HSOS/final_output.xlsx")